### Etapas iniciais: importação de pacotes, definição de funções, carregamento de dados

In [1]:
import pandas as pd

In [2]:
def values_missing(df, only_missing=True):
    '''Calcula e retorna o percentual e o número de registros nulos no dataframe df'''
    missing = pd.concat({'percent_missing': 100* df.isnull().sum() / len(df),
                         'rows_missing': df.isnull().sum()}, axis=1)
    if only_missing:
        missing = missing[missing.percent_missing > 0]
    return missing.sort_values(by='percent_missing', ascending=False)


In [3]:
winemag = pd.read_csv('datasets/winemag_reviews.csv')
winemag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149693 entries, 0 to 149692
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Title        149693 non-null  object
 1   Description  149608 non-null  object
 2   Taster       144183 non-null  object
 3   Rating       149693 non-null  int64 
 4   Price        141564 non-null  object
 5   Designation  109320 non-null  object
 6   Variety      149692 non-null  object
 7   Appellation  149693 non-null  object
 8   Winery       149693 non-null  object
 9   Alcohol      143313 non-null  object
 10  Bottle Size  149693 non-null  object
 11  Category     149693 non-null  object
 12  URL          149693 non-null  object
dtypes: int64(1), object(12)
memory usage: 14.8+ MB


### Filtragem
- O dataset originalmente continha 149.693 registros, os quais cobriam diversos tipos/categorias, como vinhos brancos, espumantes e tintos. Como o trabalho restringe-se a vinhos tintos, nessa etapa foram filtrados apenas os vinhos dessa categoria resultando em  87.574 registros.<br>

In [4]:
winemag = winemag[ winemag.Category == 'Red' ]

In [5]:
winemag.head()

,Title,Description,Taster,Rating,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,URL
0,Cantina Tollo 2015 Cagiòlo Riserva (Montepulc...,Densely packed aromas of brandy-soaked plums a...,Alexander Peartree,92,"R$ 35,00",Cagiòlo Riserva,"Montepulciano, Italian Red","Montepulciano d'Abruzzo, Central Italy, Italy",Cantina Tollo,"14,50%",750 ml,Red,https://www.winemag.com/buying-guide/cantina-t...
1,Amorotti 2016 Montepulciano d'Abruzzo,Dense aromas of black plum and blackberry are ...,Alexander Peartree,93,"R$ 44,00",NaN,"Montepulciano, Italian Red","Montepulciano d'Abruzzo, Central Italy, Italy",Amorotti,"14,00%",750 ml,Red,https://www.winemag.com/buying-guide/amorotti-...
2,Francesco Cirelli 2018 Montepulciano d'Abruzzo,"Freshly pressed blackberries, cherries and cra...",Alexander Peartree,93,"R$ 22,00",NaN,"Montepulciano, Italian Red","Montepulciano d'Abruzzo, Central Italy, Italy",Francesco Cirelli,"13,00%",750 ml,Red,https://www.winemag.com/buying-guide/francesco...
3,Terra d'Aligi 2016 Tatone (Montepulciano d'Ab...,Dense aromas of brandy-dipped cherries and bla...,Alexander Peartree,91,"R$ 17,00",Tatone,"Montepulciano, Italian Red","Montepulciano d'Abruzzo, Central Italy, Italy",Terra d'Aligi,"14,00%",750 ml,Red,https://www.winemag.com/buying-guide/terra-dal...
4,Talamonti 2017 Tre Saggi (Montepulciano d'Abr...,This lavish red is all about rich fruit and wa...,Alexander Peartree,89,"R$ 19,00",Tre Saggi,"Montepulciano, Italian Red","Montepulciano d'Abruzzo, Central Italy, Italy",Talamonti,"13,50%",750 ml,Red,https://www.winemag.com/buying-guide/talamonti...


### Tratamento de dados duplicados
 - Nenhum registro inteiramente duplicado foi encontrado;
 - Quando analisado pela coluna **Title** foram obtidos 314 registros duplicados (0.38% dos registros); desses 303 estavam duplicados também pela coluna **Taster**, indicando qu nos outros 9 casos o vinho foi objetivo foi objeto de análise por mais de um degustador; 85 dos registros duplicados possuiam também o valor do **Rating** distinto.
 - Analisando os registros duplicados no dataframe e acessando as URLs descritas no campo **url** é possível perceber que muitos dos registros duplicados devem-se a novas análises feitas sobre o vinho, seja pelo mesmo degustador ou não; na maioria dos casos a nota atribuída ao vinho permaneceu a mesma.
 - Foi optado por manter no dataset apenas o registro correspondente a análise mais recente publicada sobre o vinho, por entender que a análise mais recente é a versão mais definitiva do parecer sobre o vinho; reduzindo o número de registros para 82.260.

In [6]:
# Nenhum registro inteiramente duplicado
winemag[ winemag.duplicated() ]

,Title,Description,Taster,Rating,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,URL


In [7]:
# 314 reviews de vinho duplicados quando 
winemag.duplicated(subset=['Title']).sum()

314

In [8]:
# Exibindo parte do dataset duplicado (em conjunto com os "originais") para melhor compreender a duplicidade
winemag[ winemag.duplicated(subset=['Title'], keep=False) ].sort_values(by='Title').head(50)

,Title,Description,Taster,Rating,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,URL
31405,Ada Nada 2012 Rombone Elisa (Barbaresco),"Fig, stewed prune, sun baked soil, camphor and...",Kerin O’Keefe,92,"R$ 47,00",Rombone Elisa,Nebbiolo,"Barbaresco, Piedmont, Italy",Ada Nada,"14,50%",750 ml,Red,https://www.winemag.com/buying-guide/ada-nada-...
53182,Ada Nada 2012 Rombone Elisa (Barbaresco),This initially opens with funky aromas of barn...,Kerin O’Keefe,88,NaN,Rombone Elisa,Nebbiolo,"Barbaresco, Piedmont, Italy",Ada Nada,NaN,750 ml,Red,https://www.winemag.com/buying-guide/ada-nada-...
131487,Adega Cooperativa do Cartaxo 2015 Bridão Colhe...,Dense black fruits are the name of the game in...,Roger Voss,86,"R$ 8,00",Bridão Colheita Seleccionada,"Trincadeira, Portuguese Red","Tejo, Portugal",Adega Cooperativa do Cartaxo,"14,50%",750 ml,Red,https://www.winemag.com/buying-guide/adega-coo...
140822,Adega Cooperativa do Cartaxo 2015 Bridão Colhe...,"This wine is soft and juicy, with ripe red fru...",Roger Voss,87,"R$ 12,00",Bridão Colheita Seleccionada,"Trincadeira, Portuguese Red","Tejo, Portugal",Adega Cooperativa do Cartaxo,"14,50%",750 ml,Red,https://www.winemag.com/buying-guide/adega-coo...
32483,Adriano Marco & Vittorio 2013 Basarin (Barbar...,This opens with aromas of mature black-skinned...,Kerin O’Keefe,90,"R$ 40,00",Basarin,Nebbiolo,"Barbaresco, Piedmont, Italy",Adriano Marco & Vittorio,"14,00%",750 ml,Red,https://www.winemag.com/buying-guide/adriano-m...
109763,Adriano Marco & Vittorio 2013 Basarin (Barbar...,"Menthol, rose, wild berry, aromatic herb and n...",Kerin O’Keefe,93,"R$ 29,00",Basarin,Nebbiolo,"Barbaresco, Piedmont, Italy",Adriano Marco & Vittorio,"14,00%",750 ml,Red,https://www.winemag.com/buying-guide/adriano-m...
81734,Albert Bichot 2018 Domaine de Rochegrès (Moul...,"Property of the Bichot family, this vineyard h...",Roger Voss,93,"R$ 33,00",Domaine de Rochegrès,Gamay,"Moulin-à-Vent, Beaujolais, France",Albert Bichot,"13,00%",750 ml,Red,https://www.winemag.com/buying-guide/albert-bi...
82204,Albert Bichot 2018 Domaine de Rochegrès (Moul...,This wine comes from a five-acre vineyard in t...,Roger Voss,94,"R$ 38,00",Domaine de Rochegrès,Gamay,"Moulin-à-Vent, Beaujolais, France",Albert Bichot,"13,00%",750 ml,Red,https://www.winemag.com/buying-guide/albert-bi...
6675,Amador Cellars 2016 Estate Zinfandel (Shenando...,Very focused and delicious berry flavors give ...,Jim Gordon,92,"R$ 45,00",Estate,Zinfandel,"Shenandoah Valley (CA), Sierra Foothills, Cali...",Amador Cellars,"15,80%",750 ml,Red,https://www.winemag.com/buying-guide/amador-ce...
81357,Amador Cellars 2016 Estate Zinfandel (Shenando...,"This is a monster of a wine, very ripe, extrem...",Jim Gordon,87,"R$ 28,00",Estate,Zinfandel,"Shenandoah Valley (CA), Sierra Foothills, Cali...",Amador Cellars,"16,00%",750 ml,Red,https://www.winemag.com/buying-guide/amador-ce...


In [9]:
print('Registros duplicados com mesmo Title e Taster: {}'
      .format( winemag.duplicated(subset=['Title', 'Taster']).sum() ))
print('Registros duplicados com mesmo Title e Rating: {}'
      .format( winemag.duplicated(subset=['Title', 'Rating']).sum() ))


Registros duplicados com mesmo Title e Taster: 303
Registros duplicados com mesmo Title e Rating: 85


In [10]:
winemag = winemag.drop_duplicates(subset=['Title'], keep='first')

In [11]:
winemag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87260 entries, 0 to 149692
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        87260 non-null  object
 1   Description  87217 non-null  object
 2   Taster       84090 non-null  object
 3   Rating       87260 non-null  int64 
 4   Price        82505 non-null  object
 5   Designation  63089 non-null  object
 6   Variety      87259 non-null  object
 7   Appellation  87260 non-null  object
 8   Winery       87260 non-null  object
 9   Alcohol      82906 non-null  object
 10  Bottle Size  87260 non-null  object
 11  Category     87260 non-null  object
 12  URL          87260 non-null  object
dtypes: int64(1), object(12)
memory usage: 9.3+ MB


### Correções e transformações nos dados
- Os campos **Alcohol**, **Bottle Size** e **Price** foram convertidos de *object* para respetivamente: *float64*, *int64* e *float64*.

In [12]:
# Alcohol : transformação em número real substituindo o padrão regex [% ] por ''; e a vírgula pelo ponto.
winemag['Alcohol'] = winemag['Alcohol'].str.replace('[% ]', '', regex=True).str.replace(',', '.').astype('float64')

In [13]:
# Bottle Size : transformação em número inteiro (em ml) substituindo o padrão regex [mlML ] por ''.
print('=== Bottle Size - Valores - antes ===\n', winemag['Bottle Size'].value_counts())

winemag['Bottle Size'] = winemag['Bottle Size']\
        .str.replace('[mlML ]', '', regex=True).astype('float64')\
        .apply(lambda size: size * 1000 if size <= 3 else size).astype('int64') # Transforma L em ml

print('\n=== Bottle Size - Valores - depois ===\n', winemag['Bottle Size'].value_counts())

=== Bottle Size - Valores - antes ===
 750 ml    86423
750ML       663
3 L          64
1.5 L        39
375 ml       24
1 L          21
500 ml       15
250 ml        7
187 ml        3
1.5L          1
Name: Bottle Size, dtype: int64

=== Bottle Size - Valores - depois ===
 750     87086
3000       64
1500       40
375        24
1000       21
500        15
250         7
187         3
Name: Bottle Size, dtype: int64


In [14]:
# Price : transformação em número real substituindo o padrão regex [R$. ] por ''; e a vírgula pelo ponto.
winemag['Price'] = winemag['Price'].str.replace('[R$. ]', '', regex=True).str.replace(',', '.').astype('float64')

### Tratamento de dados omissos
- Foram encontrados valores omissos em 6 colunas e dados os seguintes tratamentos:
 - **Alcohol**: valores omissos serão tratados após a unificação do dataset com os dados do Vivino, o qual contém também informação sobre teor alcóolico.
 - **Price**: Os valores omissos foram preenchidos com o preço médio dos vinhos da mesma vinícola (Winery). Para isso foi necessário:
   - Converter o campo **Price** de *object* para *float64*.
   - Converter o campo **Bottle Size** de *object* para *float64* e tratar os valores.
   - Recalcular o preço para que todos os registros estejam no mesmo padrão de uma garrafa de 750ml (tamanho padrão). 5,5% das análises de vinho tinto publicadas no site foram feitas com tamanhos diferentes de garrafa.
   - Calcular o preço médio por vinícola e aplicar aos registros com preço faltantes.
   - Após preencher o preço médio por vinícola nos registros de preço faltantes, o número de registros omissos passou de 4755 para 910. Esses últimos valores foram preenchidos com a média geral do preço.
 - **Variety**: como apenas um registro omisso foi encontrado e não possível identificar a variedade da uva usada para a produção do vinho ao ler o review publicado no site, o registro foi descartado.
 - **Description**: foram encontrados 43 registros com a descrição do vinho nula; o campo de descrição é parte fundamental para a construção dos modelos de aprendizagem de seção 5 e, portanto, os registros nulos foram descartados.
 - **Designation**, **Taster**: valores omissos não foram tratados, pois - por trata-se de colunas não aproveitadas para a construção dos modelos preditivos, sendo a coluna **Designation** utilizada na seção 3.4 para cruzamento dos datasets.

In [15]:
values_missing(winemag)

,percent_missing,rows_missing
Designation,27.699977,24171
Price,5.449232,4755
Alcohol,4.989686,4354
Taster,3.632821,3170
Description,0.049278,43
Variety,0.001146,1


In [16]:
# Variety
winemag = winemag.dropna(subset=['Variety'])

In [17]:
# Price: O preço médio é calculado por vinícola e aplicado aos registros de preço faltante.
winemag['Price'] = winemag.groupby('Winery')['Price'].transform(lambda val: val.fillna(val.mean()))

In [18]:
values_missing(winemag)

,percent_missing,rows_missing
Designation,27.700295,24171
Alcohol,4.988597,4353
Taster,3.631717,3169
Price,1.042872,910
Description,0.049279,43


In [19]:
# Preenche os demais preços nulos do dataset com a média do preço
winemag['Price'] = winemag['Price'].fillna(winemag['Price'].mean())

In [20]:
# Elimina os registros cuja coluna de descrição seja nula
winemag.dropna(subset=['Description'], inplace=True)

In [21]:
values_missing(winemag)

,percent_missing,rows_missing
Designation,27.692167,24152
Alcohol,4.980737,4344
Taster,3.610576,3149


### Remoção de colunas
- As colunas **Category** e **URL** foram removidas do dataset. A categoria foi usada para filtrar o dataset para somente vinhos tintos, portanto pode ser removida pois os valores da coluna possuem o mesmo valor. A coluna URL foi gravada para facilitar identificar possíveis erros oriundos da coleta e não terá mais uso de agora em diante.

In [22]:
winemag = winemag.drop(['Category', 'URL'], axis=1)

In [23]:
winemag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87216 entries, 0 to 149692
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        87216 non-null  object 
 1   Description  87216 non-null  object 
 2   Taster       84067 non-null  object 
 3   Rating       87216 non-null  int64  
 4   Price        87216 non-null  float64
 5   Designation  63064 non-null  object 
 6   Variety      87216 non-null  object 
 7   Appellation  87216 non-null  object 
 8   Winery       87216 non-null  object 
 9   Alcohol      82872 non-null  float64
 10  Bottle Size  87216 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 8.0+ MB


### Exportação do Dataset

- Dos 87.574 registros de vinhos tintos inicialmente (após a filtragem), após a tratamento dos registros duplicados e omissos têm-se 87.216 registros.

In [24]:
# Exporta o dataset para arquivo csv após o tratamento dado até o momento
winemag.to_csv('datasets/winemag_red.csv', index=False, header=True)